# In This Notebook we have functions that Manipulate Data in different splits that can be used directly in Data Science Experiments and Model Training -


## 1. Splits in train and test data in such a fashion that test data sets are in different market regimes.
## 2. Second Function is that you can get any rows with specific start date and end date.
## 3. Resampling the data from 1min ohlc to 5min ohlc.

In [32]:
REPO_PATH = "/content/drive/MyDrive/Colab Notebooks/plusEV-"
csv_path = os.path.join(REPO_PATH, "nifty50_1min_2015_to_2024.csv")

In [33]:
import os
import pandas as pd

In [34]:
from google.colab import drive
drive.mount('/content/drive')
df= pd.read_csv(csv_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
df.tail()

,date,open,high,low,close,volume
890506,2024-08-28 15:25:00+05:30,25042.35,25043.85,25040.60,25041.45,0
890507,2024-08-28 15:26:00+05:30,25041.05,25043.65,25039.35,25040.30,0
890508,2024-08-28 15:27:00+05:30,25039.70,25043.25,25039.15,25042.55,0
890509,2024-08-28 15:28:00+05:30,25042.20,25042.70,25032.10,25038.90,0
890510,2024-08-28 15:29:00+05:30,25037.65,25047.65,25037.65,25044.85,0


In [36]:
import pandas as pd
import numpy as np
from typing import Tuple, List
from datetime import datetime

def split_time_series_yearly(data: pd.DataFrame, num_attempts: int = 3) -> List[Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Split time series data ensuring each year contributes one 2-month interval to test data.

    Args:
        data: DataFrame with datetime index
        num_attempts: Number of different train-test splits to create

    Returns:
        List of tuples containing (train_data, test_data) for each attempt
    """
    # Ensure data is sorted by date
    data = data.sort_index()

    # Get all unique years in the dataset
    years = sorted(data.index.year.unique())

    all_splits = []

    for attempt in range(num_attempts):
        print(f"\nAttempt {attempt + 1} intervals:")
        selected_intervals = []

        # For each year, select one random 2-month interval
        for year in years:
            # Get all dates for this year
            year_data = data[data.index.year == year]

            # Get all possible start dates for this year
            # We'll look at monthly starts to ensure clean month boundaries
            possible_starts = pd.date_range(
                start=year_data.index[0],
                end=year_data.index[-1] - pd.DateOffset(months=2),
                freq='MS'  # Month Start frequency
            )

            if len(possible_starts) > 0:
                # Randomly select a start date
                start_date = np.random.choice(possible_starts)
                end_date = start_date + pd.DateOffset(months=2) - pd.DateOffset(days=1)  # End of second month

                selected_intervals.append((start_date, end_date))
                print(f"Year {year}: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")

        # Create test dataset from selected intervals
        test_data = pd.concat([
            data[start:end]
            for start, end in selected_intervals
        ])

        # Create training dataset (all data not in test_data)
        train_data = data.drop(test_data.index)

        # Sort both datasets by date
        train_data = train_data.sort_index()
        test_data = test_data.sort_index()

        # Add to splits
        all_splits.append((train_data, test_data))

        # Print split information
        print(f"\nSplit {attempt + 1} summary:")
        print(f"Training data shape: {train_data.shape}")
        print(f"Testing data shape: {test_data.shape}")
        print("Test intervals by year:")
        for year in years:
            year_test = test_data[test_data.index.year == year]
            if not year_test.empty:
                print(f"Year {year}: {len(year_test)} days")

    return all_splits


In [37]:
# Read your Nifty50 data
df['Date'] = pd.to_datetime(df['date'])
df.set_index('Date', inplace=True)

# Get three different splits
splits = split_time_series_yearly(df)

# Save the splits if needed
# for i, (train, test) in enumerate(splits, 1):
#     train.to_csv(f'train_split_{i}.csv')
#     test.to_csv(f'test_split_{i}.csv')



Attempt 1 intervals:
Year 2015: 2015-06-01 to 2015-07-31
Year 2016: 2016-06-01 to 2016-07-31
Year 2017: 2017-04-01 to 2017-05-31
Year 2018: 2018-04-01 to 2018-05-31
Year 2019: 2019-06-01 to 2019-07-31
Year 2020: 2020-09-01 to 2020-10-31
Year 2021: 2021-03-01 to 2021-04-30
Year 2022: 2022-04-01 to 2022-05-31
Year 2023: 2023-10-01 to 2023-11-30
Year 2024: 2024-05-01 to 2024-06-30

Split 1 summary:
Training data shape: (737343, 6)
Testing data shape: (153168, 6)
Test intervals by year:
Year 2015: 16497 days
Year 2016: 15750 days
Year 2017: 14626 days
Year 2018: 15751 days
Year 2019: 15376 days
Year 2020: 16125 days
Year 2021: 14626 days
Year 2022: 14626 days
Year 2023: 14686 days
Year 2024: 15105 days

Attempt 2 intervals:
Year 2015: 2015-09-01 to 2015-10-31
Year 2016: 2016-05-01 to 2016-06-30
Year 2017: 2017-09-01 to 2017-10-31
Year 2018: 2018-09-01 to 2018-10-31
Year 2019: 2019-08-01 to 2019-09-30
Year 2020: 2020-02-01 to 2020-03-31
Year 2021: 2021-06-01 to 2021-07-31
Year 2022: 2022-0

In [38]:
15316800/737343


20.772964549741438

In [23]:
def create_named_splits(splits: List[Tuple[pd.DataFrame, pd.DataFrame]]) -> dict:
    """
    Create descriptively named DataFrames from splits

    Args:
        splits: List of (train, test) DataFrame tuples

    Returns:
        Dictionary with descriptive names and corresponding DataFrames
    """
    named_splits = {}

    for split_num, (train, test) in enumerate(splits, 1):
        # Get date ranges for train and test
        train_start = train.index.min().strftime('%Y%m%d')
        train_end = train.index.max().strftime('%Y%m%d')
        test_start = test.index.min().strftime('%Y%m%d')
        test_end = test.index.max().strftime('%Y%m%d')

        # Create descriptive names
        train_name = f"dt_train_split{split_num}_{train_start}_{train_end}"
        test_name = f"dt_test_split{split_num}_{test_start}_{test_end}"

        # Store in dictionary
        named_splits[train_name] = train
        named_splits[test_name] = test

        # Print summary
        print(f"\nSplit {split_num} DataFrames:")
        print(f"{train_name}: Shape {train.shape}")
        print(f"{test_name}: Shape {test.shape}")

    return named_splits

# Example usage:
splits = split_time_series_yearly(df, num_attempts=2)
named_dfs = create_named_splits(splits)

# To save any specific DataFrame as CSV:
# for name, df in named_dfs.items():
#     df.to_csv(f"{name}.csv")


Attempt 1 intervals:
Year 2015: 2015-05-01 to 2015-06-30
Year 2016: 2016-02-01 to 2016-03-31
Year 2017: 2017-10-01 to 2017-11-30
Year 2018: 2018-02-01 to 2018-03-31
Year 2019: 2019-05-01 to 2019-06-30
Year 2020: 2020-01-01 to 2020-02-29
Year 2021: 2021-09-01 to 2021-10-31
Year 2022: 2022-09-01 to 2022-10-31
Year 2023: 2023-05-01 to 2023-06-30
Year 2024: 2024-01-01 to 2024-02-29

Split 1 summary:
Training data shape: (737768, 6)
Testing data shape: (152743, 6)
Test intervals by year:
Year 2015: 15377 days
Year 2016: 15001 days
Year 2017: 15061 days
Year 2018: 14250 days
Year 2019: 15375 days
Year 2020: 16124 days
Year 2021: 15375 days
Year 2022: 14686 days
Year 2023: 15743 days
Year 2024: 15751 days

Attempt 2 intervals:
Year 2015: 2015-06-01 to 2015-07-31
Year 2016: 2016-06-01 to 2016-07-31
Year 2017: 2017-03-01 to 2017-04-30
Year 2018: 2018-10-01 to 2018-11-30
Year 2019: 2019-03-01 to 2019-04-30
Year 2020: 2020-10-01 to 2020-11-30
Year 2021: 2021-07-01 to 2021-08-31
Year 2022: 2022-0

In [29]:
def get_data_between_dates(df: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Extract data from DataFrame between given start and end dates (inclusive)

    Args:
        df: DataFrame with datetime index
        start_date: Start date in format 'YYYY-MM-DD'
        end_date: End date in format 'YYYY-MM-DD'

    Returns:
        DataFrame containing data between start_date and end_date
    """
    try:
        # Convert string dates to timezone-aware datetime
        start = pd.to_datetime(start_date).tz_localize('Asia/Kolkata')
        end = pd.to_datetime(end_date).tz_localize('Asia/Kolkata')

        # Verify dates are in correct order
        if start > end:
            raise ValueError("Start date cannot be after end date")

        # Filter data between dates
        mask = (df.index >= start) & (df.index <= end)
        filtered_df = df[mask]

        # Get unique dates using pandas datetime methods
        unique_dates = pd.to_datetime(filtered_df.index.date).nunique()

        # Print summary
        print(f"\nData Summary:")
        print(f"Date Range: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")
        print(f"Shape: {filtered_df.shape}")
        print(f"Number of trading days: {unique_dates}")

        return filtered_df

    except Exception as e:
        print(f"Error: {str(e)}")
        return pd.DataFrame()  # Return empty DataFrame instead of None

# Example usage:
df_slice = get_data_between_dates(df, '2023-01-01', '2023-12-31')
print("\nFirst few rows of filtered data:")
print(df_slice.head())
print(df_slice.tail())


Data Summary:
Date Range: 2023-01-01 to 2023-12-31
Shape: (91922, 6)
Number of trading days: 246

First few rows of filtered data:
                                                date      open      high  \
Date                                                                       
2023-01-02 09:15:00+05:30  2023-01-02 09:15:00+05:30  18131.70  18150.15   
2023-01-02 09:16:00+05:30  2023-01-02 09:16:00+05:30  18142.20  18145.70   
2023-01-02 09:17:00+05:30  2023-01-02 09:17:00+05:30  18133.25  18140.55   
2023-01-02 09:18:00+05:30  2023-01-02 09:18:00+05:30  18139.40  18139.40   
2023-01-02 09:19:00+05:30  2023-01-02 09:19:00+05:30  18130.30  18133.55   

                                low     close  volume  
Date                                                   
2023-01-02 09:15:00+05:30  18117.55  18141.35       0  
2023-01-02 09:16:00+05:30  18131.40  18135.65       0  
2023-01-02 09:17:00+05:30  18120.65  18140.55       0  
2023-01-02 09:18:00+05:30  18120.15  18130.75       0  

In [31]:
def resample_to_5min(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert 1-minute OHLCV data to 5-minute timeframe

    Args:
        df: DataFrame with datetime index and OHLCV columns

    Returns:
        DataFrame with 5-minute resampled data
    """
    try:
        # Make a copy to avoid modifying original data
        df_5min = df.copy()

        # Resample to 5-minute intervals using 'min' instead of 'T'
        df_5min = df_5min.resample('5min').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        })

        # Remove any rows with NaN values
        df_5min = df_5min.dropna()

        # Get unique dates properly
        unique_dates = len(set(df_5min.index.date))

        # Print summary
        print("\nResampling Summary:")
        print(f"Original shape (1min): {df.shape}")
        print(f"Resampled shape (5min): {df_5min.shape}")
        print(f"Time range: {df_5min.index.min()} to {df_5min.index.max()}")
        print(f"Number of trading days: {unique_dates}")

        # Print first few rows to verify data
        print("\nFirst few rows of 5-minute data:")
        print(df_5min.head())

        return df_5min

    except Exception as e:
        print(f"Error during resampling: {str(e)}")
        return pd.DataFrame()

# Example usage:
df_5min = resample_to_5min(df)
print("\nFirst few rows of 5-minute data:")
print(df_5min.head())


Resampling Summary:
Original shape (1min): (890511, 6)
Resampled shape (5min): (178118, 5)
Time range: 2015-01-09 09:15:00+05:30 to 2024-08-28 15:25:00+05:30
Number of trading days: 2385

First few rows of 5-minute data:
                              open     high      low    close  volume
Date                                                                 
2015-01-09 09:15:00+05:30  8285.45  8301.30  8285.45  8301.20       0
2015-01-09 09:20:00+05:30  8300.50  8303.00  8293.25  8301.00       0
2015-01-09 09:25:00+05:30  8301.65  8302.55  8286.80  8294.15       0
2015-01-09 09:30:00+05:30  8294.10  8295.75  8280.65  8288.50       0
2015-01-09 09:35:00+05:30  8289.10  8290.45  8278.00  8283.45       0

First few rows of 5-minute data:
                              open     high      low    close  volume
Date                                                                 
2015-01-09 09:15:00+05:30  8285.45  8301.30  8285.45  8301.20       0
2015-01-09 09:20:00+05:30  8300.50  8303.00 

In [44]:
### git ### push the changes


!ls

drive  sample_data  train_split_1.csv
